In [1]:
import sys
import hydra
from hydra import initialize_config_dir, compose, initialize
import torch
import lightning as L
import matplotlib.pyplot as plt
import random
import numpy as np
from pathlib import Path
import ngssf
import cv2
import open3d as o3d
import trimesh
from tqdm.notebook import tqdm
from ngssf.sampler import FieldDataGenerator
from ngssf.util import _spectrum, VideoEncoderThread

sys.path.append("/Code/viscomp_reconstruction/ngssf/scripts")
from train_hydra import Trainer

# -------------------------------------------------------------------------------
# Variables
# initialize(version_base=None, config_path="../configs")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# -------------------------------------------------------------------------------
# Functions to load the model
def get_cfg_from_experiment(experiment_path: Path, overrides: list):
    # Point to the .hydra folder you used during training
    hydra_cfg_dir = str(experiment_path / ".hydra")

    with initialize_config_dir(version_base=None, config_dir=hydra_cfg_dir):
        # config_name is the filename without “.yaml”
        cfg = compose(config_name="config", overrides=overrides)
    return cfg
# -------------------------------------------------------------------------------
# Load the kernels for visualizations
# Load anisotropic kernels from the paper
anisotropic_paper = torch.load(ngssf.data._data_dir / "covariance_matrices" / "3d.pt").to(device)
# anisotropic_paper = torch.load(ngssf.data._data_dir / "covariance_matrices" / "2d_vis.pt")['video'].to(device)

# # Generate isotropic kernels
# # isotropic_kernels = ngssf.data.benchmark_variances()
# isotropic_kernels = torch.tensor([1e-11, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2])
# isotropic_kernels = torch.eye(anisotropic_paper.shape[-1])[None] * isotropic_kernels[:, None, None]
# isotropic_kernels = isotropic_kernels.to(device)

# # Generate anisotropic kernels
# anisotropic_small = FieldDataGenerator._rand_covariance_matrices_with_logrand_eigenvalues(
#     torch.tensor([-9.0, -4.8], device=device), torch.tensor([-6.0, -4.0], device=device), 
#     2, 100, 
# device=device)
# anisotropic_medium = FieldDataGenerator._rand_covariance_matrices_with_logrand_eigenvalues(
#     torch.tensor([-8.0, -4.0], device=device), torch.tensor([-5.0, -3.0], device=device), 
#     2, 100,
# device=device)
# anisotropic_large = FieldDataGenerator._rand_covariance_matrices_with_logrand_eigenvalues(
#     torch.tensor([-7.0, -3.0], device=device), torch.tensor([-4.0, -2.2], device=device), 
#     2, 100,
# device=device)

# Generate isotropic kernels from the paper
isotropic_kernels_list_paper = [1e-12, 0.0001, 0.001, 0.01, 0.1]
isotropic_kernels_paper = torch.tensor(isotropic_kernels_list_paper)
isotropic_kernels_paper = torch.eye(anisotropic_paper.shape[-1])[None] * isotropic_kernels_paper[:, None, None]
isotropic_kernels_paper = isotropic_kernels_paper.to(device, dtype=torch.float32)

# Generate video kernels
video_kernels = ngssf.util._interpolate_2d_covariance_matrices(
    np.array([0, 25, 100, 150, 200, 230, 280, 310, 360, 438]),
    np.array([
        [0, -7, -7], [0, -4, -4], [0, -1, -1], [0, -4, -1], [0.125, -4, -1], 
        [0.125, -4, -2], [0.25, -4, -2], [0.25, -4, -3], [0.5, -4, -3], [1, -7, -7],
    ])
)

# # 3D video kernels
# times = np.array([0, 180, 300, 420, 540])
# logvars = np.array([[-7, -7, -7], [-3, -3, -3], [-3, -7, -3], [-7, -3, -7], [-7, -7, -7]])
# frames = np.arange(times.max() + 1)
# vis_kernels_3d = np.zeros((len(frames), 3, 3))
# vis_kernels_3d[:, 0, 0] = 10 ** np.interp(frames, times, logvars[:, 0])
# vis_kernels_3d[:, 1, 1] = 10 ** np.interp(frames, times, logvars[:, 1])
# vis_kernels_3d[:, 2, 2] = 10 ** np.interp(frames, times, logvars[:, 2])
# vis_kernels_3d = torch.tensor(vis_kernels_3d, device=device, dtype=torch.float32)

# # 3D Isotropic kernels
# isotropic_kernels_3d = torch.tensor([1e-12, 0.0001, 0.001, 0.01, 0.1], dtype=torch.float32, device="cuda")
# isotropic_kernels_3d = torch.eye(3, device="cuda")[None] * isotropic_kernels_3d[:, None, None]

# # Save the kernels
# testing_kernels = {
#     # "isotropic": isotropic_kernels.to("cpu")
#     # "original_only": isotropic_kernels_paper[0][None].to("cpu"),
# }
# for i, kernel in enumerate(isotropic_kernels_paper):
#     testing_kernels[f"isotropic_paper_{isotropic_kernels_list_paper[i]}"] = kernel[None].to("cpu", dtype=torch.float32)
    
# testing_kernels |= {
#     # "anisotropic_small": anisotropic_small.to("cpu"),
#     # "anisotropic_medium": anisotropic_medium.to("cpu"),
#     # "anisotropic_large": anisotropic_large.to("cpu"),
#     "anisotropic_paper": anisotropic_paper.to("cpu", dtype=torch.float32),
#     # "video": cov_mats.to("cpu")
# }
# torch.save({"video": vis_kernels_3d}, ngssf.data._data_dir / "covariance_matrices" / "3d_vis.pt")
# torch.save({"isotropic": isotropic_kernels_3d}, ngssf.data._data_dir / "covariance_matrices" / "3d_isotropic.pt")
# torch.save(testing_kernels, ngssf.data._data_dir / "covariance_matrices" / "3d_paper.pt")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/tmp/ipykernel_7946/2847699351.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  anisotropic_paper = torch.load(ngssf.data._data_dir / "covariance_matrices" / "3d.pt").to

In [ ]:
# experiments_root_path = Path("/Code/viscomp_reconstruction/Data/Experiments/ngssf/sdf_experiments")
experiments_root_path = Path("/Code/viscomp_reconstruction/Data/Experiments/ngssf/family_experiments")

# experiment_relative_path1 = "background_anisotropic_gaussian/background-ngssf-gaussian-alien"
# experiment_relative_path1 = "sdf-ours-gaussian-dragon"
experiment_relative_path1 = "ngssf-1mc-nolipschitz-gaussian-curiouscat"
experiment_path1 = experiments_root_path / experiment_relative_path1
cfg1 = get_cfg_from_experiment(
    experiment_path=experiment_path1,
    overrides=[
        f"paths.output_dir={str(experiment_path1)}",
        "tensorboard=null",
        "ckpt_path=tmp",
    ],
)

# cfg1.dataset.field.source_field.voxel_resolution=256
# cfg1.dataset.field.source_field.input_coord_cache = 0
# cfg1.dataset.field.source_field.voxel_resolution = 1024
trainer = Trainer(cfg1).to(device)
_ = trainer.eval()

# experiment_relative_path2 = "ngssf_pretrained/ngssf-orig-picture-apples"
# experiment_path2 = experiments_root_path / experiment_relative_path2
# cfg2 = get_cfg_from_experiment(
#     experiment_path=experiment_path2,
#     overrides=[
#         f"paths.output_dir={str(experiment_path1)}",
#         "tensorboard=null",
#         "ckpt_path=null",
#     ],
#     device=device
# )
# trainer2 = Trainer(cfg2).to(device)
# trainer2.load_state_dict_from_path(str(experiment_path2), load_training_state=False)

Seed set to 42
/opt/conda/lib/python3.10/site-packages/torchmetrics/functional/image/lpips.py:332: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.l

In [ ]:
import os, uuid, tempfile, numpy as np, cv2, trimesh, open3d as o3d
import blendertoolbox as bt
import os, math, bpy

def _interpolate_3d_covariance_matrices(times, angles_xyz_and_logvars):
    """
    Interpolate per-frame 3D covariance matrices from (angles, log-variances).

    Parameters
    ----------
    times : 1D array-like of shape (K,)
        Keyframe time indices (must be non-decreasing for np.interp).
    angles_xyz_and_logvars : array-like of shape (K, 6)
        Columns: [ax, ay, az, lvx, lvy, lvz]
        - ax, ay, az are rotations about X/Y/Z respectively, given on [0,1) and
          internally scaled to radians via angle * 2π.
        - lvx, lvy, lvz are base-10 log-variances along the principal axes.

    Returns
    -------
    torch.Tensor of shape (T, 3, 3) on CUDA
        Full covariance per frame t = 0..max(times):
            Σ_t = R_t @ diag(10**lvx_t, 10**lvy_t, 10**lvz_t) @ R_t^T
        Rotation convention: R = Rz(az) @ Ry(ay) @ Rx(ax)  (yaw-pitch-roll).
    """
    times = np.asarray(times)
    frames = np.arange(int(times.max()) + 1)

    ax = np.interp(frames, times, angles_xyz_and_logvars[:, 0])
    ay = np.interp(frames, times, angles_xyz_and_logvars[:, 1])
    az = np.interp(frames, times, angles_xyz_and_logvars[:, 2])
    lvx = np.interp(frames, times, angles_xyz_and_logvars[:, 3])
    lvy = np.interp(frames, times, angles_xyz_and_logvars[:, 4])
    lvz = np.interp(frames, times, angles_xyz_and_logvars[:, 5])

    # Scale normalized angles to radians
    ax = ax * (2 * np.pi)
    ay = ay * (2 * np.pi)
    az = az * (2 * np.pi)

    cx, sx = np.cos(ax), np.sin(ax)
    cy, sy = np.cos(ay), np.sin(ay)
    cz, sz = np.cos(az), np.sin(az)

    # R = Rz(az) @ Ry(ay) @ Rx(ax) ; explicit batched elements
    r00 = cz * cy
    r01 = cz * sy * sx - sz * cx
    r02 = cz * sy * cx + sz * sx

    r10 = sz * cy
    r11 = sz * sy * sx + cz * cx
    r12 = sz * sy * cx - cz * sx

    r20 = -sy
    r21 = cy * sx
    r22 = cy * cx

    rot_mats = np.stack(
        [
            np.stack([r00, r01, r02], axis=-1),
            np.stack([r10, r11, r12], axis=-1),
            np.stack([r20, r21, r22], axis=-1),
        ],
        axis=-2,  # shape (T, 3, 3)
    )

    var_mats = np.zeros((len(frames), 3, 3), dtype=np.float64)
    var_mats[:, 0, 0] = 10.0 ** lvx
    var_mats[:, 1, 1] = 10.0 ** lvy
    var_mats[:, 2, 2] = 10.0 ** lvz

    covs = rot_mats @ var_mats @ np.swapaxes(rot_mats, 1, 2)
    return torch.as_tensor(covs, dtype=torch.float32, device="cuda")

def get_gt_pred_with_insets(batch, outputs):
    gt, pred = batch["rgb"], outputs["rgb"]  # HxWx3, [-1,1], torch

    def to_bgr_u8_and_01(x):
        x01 = (x + 1.0) / 2.0
        bgr = (x01.clamp(0,1) * 255).round().to(torch.uint8).cpu().numpy()[..., ::-1]
        return bgr, x01.clamp(0,1)

    gt_bgr, gt01   = to_bgr_u8_and_01(gt)
    pred_bgr, pr01 = to_bgr_u8_and_01(pred)

    H, W = pred_bgr.shape[:2]
    m = max(2, int(min(H, W) * 0.02))      # margin
    b = max(1, int(min(H, W) * 0.004))     # border
    side = min(int((H - 3*m)//2 - 2*b), int(W * 0.30))
    ih = side + 2*b                        # inset height with border

    def place(img, ins, y):
        ins = cv2.resize(ins, (side, side), interpolation=cv2.INTER_AREA)
        ins = cv2.copyMakeBorder(ins, b, b, b, b, cv2.BORDER_CONSTANT, value=(255,255,255))
        x = W - ins.shape[1] - m
        img[y:y+ins.shape[0], x:x+ins.shape[1]] = ins

    # --- error inset (×4 to dramatize), top-right ---
    err = ((pr01 - gt01).abs().mean(-1) * 4.0).clamp(0,1).cpu().numpy()
    err_u8  = (err * 255).round().astype(np.uint8)
    cmap = cv2.COLORMAP_MAGMA if hasattr(cv2, "COLORMAP_MAGMA") else cv2.COLORMAP_INFERNO
    err_vis = cv2.applyColorMap(err_u8, cmap)

    # --- spectrum inset helper (expects CHW, returns [-1,1]) ---
    def spectrum_vis(x_rgb_m1p1):
        s = _spectrum(x_rgb_m1p1.permute(2,0,1).cpu())
        if isinstance(s, torch.Tensor): s = s.detach().cpu()
        s = np.asarray(s.squeeze())                         # HxW in [-1,1]
        u8 = np.clip((s + 1.0) * 127.5, 0, 255).round().astype(np.uint8)
        return cv2.cvtColor(u8, cv2.COLOR_GRAY2BGR)

    spec_pred = spectrum_vis(pred)
    spec_gt   = spectrum_vis(gt)

    # --- compose and save ---
    out_pred = pred_bgr.copy()
    place(out_pred, err_vis, m)                 # top-right
    place(out_pred, spec_pred, H - ih - m)      # bottom-right
    out_gt = gt_bgr.copy()
    place(out_gt, spec_gt, H - ih - m)          # bottom-right

    return out_gt, out_pred




def render_ply(ply_filename, outputPath, is_lucy=False, use_vert_color=False,
               imgRes_x=1024, imgRes_y=1024, samples_if_textured=100, samples_if_vcol=20,
               exposure=1.5, use_GPU=True, blend_name='test.blend'):
    # Initialize blender
    numSamples = samples_if_vcol if use_vert_color else samples_if_textured
    bt.blenderInit(imgRes_x, imgRes_y, numSamples, exposure, use_GPU)

    # Load the blend file
    cwd = os.getcwd()
    blend_file = os.path.join(cwd, blend_name)
    with bpy.data.libraries.load(blend_file, link=False) as (data_from, data_to):
        data_to.objects = data_from.objects

    # Link all objects
    for obj in data_to.objects:
        if obj is not None:
            bpy.context.scene.collection.objects.link(obj)

    # Reference GT object (for transform/material)
    gt_obj = bpy.data.objects.get('gt')
    if gt_obj is None:
        raise SystemExit('GT object not found in the blend file.')

    # Store transform + material then hide GT
    location = tuple(gt_obj.location)
    rotation = tuple(math.degrees(r) for r in gt_obj.rotation_euler)
    scale    = tuple(gt_obj.scale)
    gt_mat   = gt_obj.active_material
    gt_obj.hide_viewport = True
    gt_obj.hide_render   = True

    # Import new mesh at GT transform
    ply_path = os.path.join(cwd, ply_filename)
    if not os.path.exists(ply_path):
        raise SystemExit(f'PLY file not found: {ply_path}')
    new_mesh = bt.readMesh(ply_path, location, rotation, scale)

    # Shading
    if use_vert_color:
        # emission from vertex colors (layer name "Col" on PLY import)
        mat = bpy.data.materials.new(name="VertexColorFlatMaterial")
        mat.use_nodes = True
        nodes = mat.node_tree.nodes; nodes.clear()
        vc_node  = nodes.new(type="ShaderNodeVertexColor"); vc_node.layer_name = "Col"
        emission = nodes.new(type="ShaderNodeEmission")
        output   = nodes.new(type="ShaderNodeOutputMaterial")
        mat.node_tree.links.new(vc_node.outputs['Color'], emission.inputs['Color'])
        mat.node_tree.links.new(emission.outputs['Emission'], output.inputs['Surface'])

        if new_mesh.data.materials:
            new_mesh.data.materials[0] = mat
        else:
            new_mesh.data.materials.append(mat)

        # Remove ground so it doesn’t tint the inset render
        plane = bpy.data.objects.get('Plane')
        if plane:
            bpy.ops.object.select_all(action='DESELECT')
            plane.select_set(True)
            bpy.context.view_layer.objects.active = plane
            bpy.ops.object.delete()
    else:
        bpy.ops.object.shade_smooth()
        if gt_mat:
            new_mesh.active_material = gt_mat

    cam = bpy.data.objects.get('Camera')
    if cam is None:
        raise SystemExit('Camera not found in the blend file.')

    if is_lucy:
        new_mesh.rotation_euler[0] = 0

    # Save updated blend (optional)
    # bpy.ops.wm.save_mainfile(filepath=os.path.join(cwd, 'test_updated.blend'))

    bt.renderImage(outputPath, cam)
    return outputPath


def _place_top_right_inset(base_bgr, inset_bgr, margin_frac=0.02, border_frac=0.004):
    H, W = base_bgr.shape[:2]
    m = max(2, int(min(H, W) * margin_frac))
    b = max(1, int(min(H, W) * border_frac))
    side = min(int((H - 3*m)//2 - 2*b), int(W * 0.30))
    ins = cv2.resize(inset_bgr, (side, side), interpolation=cv2.INTER_AREA)
    ins = cv2.copyMakeBorder(ins, b, b, b, b, cv2.BORDER_CONSTANT, value=(255,255,255))
    out = base_bgr.copy()
    x = W - ins.shape[1] - m; y = m
    out[y:y+ins.shape[0], x:x+ins.shape[1]] = ins
    return out

def get_gt_pred_with_insets_sdf(gt_tm: trimesh.Trimesh,
                                pred_tm: trimesh.Trimesh,
                                resolution=1024,
                                err_mult=1.0,
                                clip_percentile=95,
                                is_lucy=False,
                                blend_name='test.blend'):
    """
    Returns: (gt_bgr, pred_with_error_inset_bgr), both at `resolution`×`resolution`.
    Renders are produced by Blender; error uses Open3D SDF (pred→gt), colored with MAGMA.
    """

    # --- 1) SDF error with Open3D (pred→gt) ---
    def _tm_to_o3d(m: trimesh.Trimesh):
        o = o3d.geometry.TriangleMesh()
        o.vertices = o3d.utility.Vector3dVector(m.vertices.astype(np.float64))
        o.triangles = o3d.utility.Vector3iVector(m.faces.astype(np.int32))
        return o

    scene = o3d.t.geometry.RaycastingScene()
    _ = scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(_tm_to_o3d(gt_tm)))
    pred_pts_legacy = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pred_tm.vertices.astype(np.float64)))
    pred_pts_t = o3d.t.geometry.PointCloud.from_legacy(pred_pts_legacy)
    sdf = scene.compute_signed_distance(pred_pts_t.point.positions).numpy()  # (N,)

    err = np.abs(sdf) * float(err_mult)
    if clip_percentile is not None:
        mx = np.percentile(err, clip_percentile); mx = mx if mx > 1e-12 else 1.0
        err01 = np.clip(err / mx, 0.0, 1.0)
    else:
        err01 = np.clip(err, 0.0, 1.0)
    err_u8 = (err01 * 255).astype(np.uint8)

    # Color with MAGMA
    cmap = cv2.COLORMAP_MAGMA if hasattr(cv2, "COLORMAP_MAGMA") else cv2.COLORMAP_INFERNO
    col_bgr = cv2.applyColorMap(err_u8[:, None], cmap)[:, 0, :]     # Nx3
    col_rgb = col_bgr[:, ::-1]
    col_rgba = np.concatenate([col_rgb, 255*np.ones((col_rgb.shape[0], 1), dtype=np.uint8)], axis=1)

    pred_col = pred_tm.copy()
    pred_col.visual.vertex_colors = col_rgba  # per-vertex RGBA (uint8)

    # --- 2) Export temp PLYs for Blender ---
    tmpdir = tempfile.mkdtemp(prefix="meshviz_")
    fn_gt   = os.path.join(tmpdir, f"gt_{uuid.uuid4().hex}.ply")
    fn_pred = os.path.join(tmpdir, f"pred_{uuid.uuid4().hex}.ply")  # colored

    gt_tm.export(fn_gt)
    pred_col.export(fn_pred)  # carries vertex colors for the error render

    # --- 3) Blender renders (plain GT, plain Pred, error Pred via vertex colors) ---
    fn_img_gt   = os.path.join(tmpdir, f"gt_{uuid.uuid4().hex}.png")
    fn_img_pred = os.path.join(tmpdir, f"pred_{uuid.uuid4().hex}.png")
    fn_img_err  = os.path.join(tmpdir, f"pred_err_{uuid.uuid4().hex}.png")

    # GT: textured/smooth (no vertex color)
    render_ply(fn_gt,   fn_img_gt,   is_lucy=is_lucy, use_vert_color=False,
               imgRes_x=resolution, imgRes_y=resolution, blend_name=blend_name)

    # PRED base: use the same PLY but ignore vertex colors (material shades it)
    render_ply(fn_pred, fn_img_pred, is_lucy=is_lucy, use_vert_color=False,
               imgRes_x=resolution, imgRes_y=resolution, blend_name=blend_name)

    # PRED error: emission from vertex colors
    render_ply(fn_pred, fn_img_err,  is_lucy=is_lucy, use_vert_color=True,
               imgRes_x=resolution, imgRes_y=resolution, blend_name=blend_name)

    # --- 4) Compose inset and return frames (BGR) ---
    gt_bgr   = cv2.imread(fn_img_gt,  cv2.IMREAD_COLOR)
    pred_bgr = cv2.imread(fn_img_pred, cv2.IMREAD_COLOR)
    err_bgr  = cv2.imread(fn_img_err,  cv2.IMREAD_COLOR)

    pred_with_inset = _place_top_right_inset(pred_bgr, err_bgr)
    return gt_bgr, pred_with_inset

could not get a list of mounted file-systems


In [ ]:
# anisotropic_sdf = _interpolate_3d_covariance_matrices(
#     [0, 15, 65, 100, 135, 155, 190, 210, 245, 299],
#     np.array([
#         [0, 0, 0, -7, -7, -7], 
#         [0, 0, 0, -4, -4, -4], 
#         [0, 0, 0, -3, -3, -3], 
#         [0, 0, 0, -4, -1, -4], 
#         [0.125, 0.0, 0.0, -4, -1, -4], 
#         [0.125, 0.0, 0.0, -4, -3, -4], 
#         [0.25, 0.25, 0.0, -4, -3, -3], 
#         [0.25, 0.25, 0.0, -4, -3, -3], 
#         [0.5, 0.25, 0.25, -4, -3, -3], 
#         [1, 0.0, 0.0, -7, -7, -7]
#     ])
# )

# for kernel in ["gaussian", "uniform_ellipsoid", "lanczos"]:
#     trainer.model.encoder.smoothing_mode = kernel
#     trainer.dataset.field._sampling_mode = kernel
    
#     # gt_recorder = VideoEncoderThread(
#     #     f"output/gt_{kernel}.mp4",
#     #     fps=30,
#     #     width=resolution,
#     #     height=resolution,
#     #     options={
#     #         "preset": "slow",       # Slower encoding yields better compression efficiency
#     #         "crf": "23",            # Near visually lossless quality (experiment with higher values if acceptable)
#     #         "tune": "stillimage",   # Optimizes compression for nearly static content
#     #         "sc_threshold": "0",    # Disables scene cut detection to enforce a fixed GOP
#     #     }
#     # )
#     # pred_recorder = VideoEncoderThread(
#     #     f"output/pred_{kernel}.mp4",
#     #     fps=30,
#     #     width=resolution,
#     #     height=resolution,
#     #     options={
#     #         "preset": "slow",       # Slower encoding yields better compression efficiency
#     #         "crf": "23",            # Near visually lossless quality (experiment with higher values if acceptable)
#     #         "tune": "stillimage",   # Optimizes compression for nearly static content
#     #         "sc_threshold": "0",    # Disables scene cut detection to enforce a fixed GOP
#     #     }
#     # )
#     # gt_recorder.start()
#     # pred_recorder.start()
    
#     for index in tqdm(range(len(anisotropic_sdf))):
#         index = 100
#         scale = anisotropic_sdf[[index]]
    
#         # Get the test data
#         batch = trainer.dataset.get_test_data(
#             scale,
#             cfg1.trainer.testing_resolution,
#             batch_size=trainer.cfg.trainer.testing_batch_size,
#         )
    
#         # Run the model
#         # Perform the testing step
#         with torch.no_grad():
#             outputs = trainer.run_batched(batch, trainer.cfg.trainer.testing_batch_size)
#             # Calculate the metrics
#             loss, metrics = trainer.model.calculate_loss_and_log(
#                 batch, outputs, trainer, log_mode="test_with_vis")
    
#         out_gt, out_pred = get_gt_pred_with_insets_sdf(
#             metrics['mesh']['gt'], metrics['mesh']['pred'])

#         break
#         # gt_recorder.enqueue_frame(out_gt)
#         # pred_recorder.enqueue_frame(out_pred)
        
#     break
#     # gt_recorder.stop()
#     # pred_recorder.stop()

In [11]:
resolution = 2048
kernels_image = ngssf.util._interpolate_2d_covariance_matrices(
    np.array([0, 15, 55, 85, 115, 130, 160, 175, 205, 249]),
    np.array([
        [0, -7, -7], [0, -4, -4], [0, -1, -1], [0, -4, -1], [0.125, -4, -1], 
        [0.125, -3, -2], [0.25, -3, -2], [0.25, -4, -3], [0.5, -4, -3], [1, -7, -7],
    ])
).cuda()


for kernel in ["gaussian", "uniform_ellipsoid", "lanczos"]:
    trainer.model.encoder.smoothing_mode = kernel
    trainer.dataset.field._sampling_mode = kernel
    
    gt_recorder = VideoEncoderThread(
        f"output/gt_{kernel}.mp4",
        fps=25,
        width=resolution,
        height=resolution,
        options={
            "preset": "slow",       # Slower encoding yields better compression efficiency
            "crf": "23",            # Near visually lossless quality (experiment with higher values if acceptable)
            "tune": "stillimage",   # Optimizes compression for nearly static content
            "sc_threshold": "0",    # Disables scene cut detection to enforce a fixed GOP
        }
    )
    pred_recorder = VideoEncoderThread(
        f"output/pred_{kernel}.mp4",
        fps=25,
        width=resolution,
        height=resolution,
        options={
            "preset": "slow",       # Slower encoding yields better compression efficiency
            "crf": "23",            # Near visually lossless quality (experiment with higher values if acceptable)
            "tune": "stillimage",   # Optimizes compression for nearly static content
            "sc_threshold": "0",    # Disables scene cut detection to enforce a fixed GOP
        }
    )
    kernel_recorder = VideoEncoderThread(
        f"output/kernel_{kernel}.mp4",
        fps=25,
        width=1024,
        height=1024,
        options={
            "preset": "slow",       # Slower encoding yields better compression efficiency
            "crf": "23",            # Near visually lossless quality (experiment with higher values if acceptable)
            "tune": "stillimage",   # Optimizes compression for nearly static content
            "sc_threshold": "0",    # Disables scene cut detection to enforce a fixed GOP
        }
    )
    gt_recorder.start()
    pred_recorder.start()
    kernel_recorder.start()
    
    for index in tqdm(range(len(kernels_image))):
        scale = kernels_image[[index]]
    
        # Get the test data
        batch = trainer.dataset.get_test_data(
            scale,
            resolution,
            batch_size=trainer.cfg.trainer.testing_batch_size,
        )
    
        # Run the model
        # Perform the testing step
        with torch.no_grad():
            outputs = trainer.run_batched(batch, trainer.cfg.trainer.testing_batch_size)
            # Calculate the metrics
            # loss, metrics = trainer.model.calculate_loss_and_log(
            #     batch, outputs, trainer, log_mode="test_with_vis")
    
        out_gt, out_pred = get_gt_pred_with_insets(
            batch, outputs)
        gt_recorder.enqueue_frame(out_gt)
        pred_recorder.enqueue_frame(out_pred)
    
        kernel_vis = render_kernel(
            kernel, scale, interval=(-0.5, 0.5),
        )
        kernel_recorder.enqueue_frame(kernel_vis)
    
    gt_recorder.stop()
    pred_recorder.stop()
    kernel_recorder.stop()

  0%|          | 0/250 [00:00<?, ?it/s]

W1009 23:51:17.944000 139705020405568 torch/fx/experimental/symbolic_shapes.py:4449] [0/0] xindex is not in var_ranges, defaulting to unknown range.
W1009 23:51:18.662000 139705020405568 torch/fx/experimental/symbolic_shapes.py:4449] [0/1] xindex is not in var_ranges, defaulting to unknown range.


  0%|          | 0/250 [00:00<?, ?it/s]

W1010 00:41:10.170000 139705020405568 torch/fx/experimental/symbolic_shapes.py:4449] [0/2] xindex is not in var_ranges, defaulting to unknown range.


  0%|          | 0/250 [00:00<?, ?it/s]

W1010 01:34:40.139000 139705020405568 torch/fx/experimental/symbolic_shapes.py:4449] [2/0] xindex is not in var_ranges, defaulting to unknown range.
W1010 01:34:41.659000 139705020405568 torch/fx/experimental/symbolic_shapes.py:4449] [2/1] xindex is not in var_ranges, defaulting to unknown range.


In [ ]:
# trainer = trainer1

# trainer.eval()

# # Select the kernel
# index = 1
# scale = anisotropic_paper[[index]]
# # scale = torch.eye(3, device=device)[None] * 1e-6

# print(scale)

# # Get the test data
# batch = trainer.dataset.get_test_data(
#     scale,
#     256,
#     batch_size=trainer.cfg.trainer.testing_batch_size,
# )

# # Run the model
# # Perform the testing step
# with torch.no_grad():
#     outputs = trainer.run_batched(batch, trainer.cfg.trainer.testing_batch_size)
#     # Calculate the metrics
#     loss, metrics = trainer.model.calculate_loss_and_log(
#         batch, outputs, trainer, log_mode="test_with_vis")
    
# if "vis" in metrics:
#     vis = metrics.pop("vis")
#     fig = plt.figure(figsize=(12, 12))
#     plt.imshow(vis[..., [2,1,0]])
#     plt.axis("off")